In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from dataset import KITTIDataset
import tqdm
from matplotlib import pyplot as plt
import numpy as np
import time
from model import SparseConv2d,MyFCN,FCN32s,MySparseFCN
batch_size = 8
test_data = KITTIDataset(num=0,file="val")
test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=True)
criterion = torch.nn.L1Loss(reduction='mean')

model = MyFCN(in_channel=1,n_class=1)
model.load_state_dict(torch.load(f"model/{model.__class__.__name__}_epoch20.pt"))
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
losses = []
model.eval()
model.to(device)
with torch.no_grad():
    bar = tqdm.tqdm(range(len(test_dataloader)))
    for feature,groundtruth in test_dataloader:
        feature = feature.to(device)
        groundtruth = groundtruth.to(device)
        loss = criterion(model(feature),groundtruth)
        losses.append(loss.item())
        bar.update()
    bar.close()

100%|██████████| 429/429 [03:37<00:00,  1.98it/s]


In [3]:
print(f"MAE in test dataset of {len(test_data)} imgs:{np.array(losses).mean()}")

MAE in test dataset of 3426 imgs:4.646885761847863


In [4]:
index = test_data.index_threshold.copy()
index.pop()
index.insert(0,0)
for i,sync in zip(index,test_data.num_imgs_sync.keys()):
    print(f"Index:{i}\tsync:{sync}\tShape:{test_data[i][0].shape}")

Index:0	sync:2011_09_26_drive_0002_sync	Shape:torch.Size([1, 370, 1224])
Index:67	sync:2011_09_26_drive_0005_sync	Shape:torch.Size([1, 370, 1224])
Index:211	sync:2011_09_26_drive_0013_sync	Shape:torch.Size([1, 370, 1224])
Index:345	sync:2011_09_26_drive_0020_sync	Shape:torch.Size([1, 370, 1224])
Index:421	sync:2011_09_26_drive_0023_sync	Shape:torch.Size([1, 370, 1224])
Index:885	sync:2011_09_26_drive_0036_sync	Shape:torch.Size([1, 370, 1224])
Index:1678	sync:2011_09_26_drive_0079_sync	Shape:torch.Size([1, 370, 1224])
Index:1768	sync:2011_09_26_drive_0095_sync	Shape:torch.Size([1, 370, 1224])
Index:2026	sync:2011_09_26_drive_0113_sync	Shape:torch.Size([1, 370, 1224])
Index:2103	sync:2011_09_28_drive_0037_sync	Shape:torch.Size([1, 370, 1224])
Index:2182	sync:2011_09_29_drive_0026_sync	Shape:torch.Size([1, 370, 1224])
Index:2330	sync:2011_09_30_drive_0016_sync	Shape:torch.Size([1, 370, 1224])
Index:2599	sync:2011_10_03_drive_0047_sync	Shape:torch.Size([1, 370, 1224])
